# Introduction to Property Value Management (PVMT)

This notebook present the current capabilities of the PVMT extension for py-capellambse.
We will see how to display all defined property value domains, their groups and the
available properties.
Further we'll see how to access applied property values from ModelObjects. This is demonstrated on the list of LogicalActors.

In [1]:
import capellambse


path_to_model = "../tests/data/melodymodel/5_2/Melody Model Test.aird"
model = capellambse.MelodyModel(path_to_model, jupyter_untrusted=True)

You can investigate all defined PVMT domains, their groups and properties:

In [10]:
darkmagic = model.property_value_packages[0].packages[0]
display(darkmagic)
display(darkmagic.groups)
for grp in darkmagic.groups:
    display(grp.name)
    display(grp.values)

constraints,(Empty list)
description,None
diagrams,(Empty list)
enumeration_property_types,"EnumerationPropertyType ""DarkMagicPowerEnum"" (2644e99b-72c8-42f0-9bba-5141dda592e9)"
groups,"PropertyValueGroup ""Power"" (1e241678-282c-46f3-abbd-e0cf34e743ee)PropertyValueGroup ""Power Level"" (a0cb5a23-955e-43d6-a633-2c4e66991364)"
name,DarkMagic
packages,(Empty list)
parent,"PropertyValuePkg ""EXTENSIONS"" (fdbeb3b8-e1e2-4c6d-aff4-bccb1b9f437f)"
progress_status,NOT_SET
pvmt,<capellambse.extensions.pvmt.PropertyValueProxy object at 0x7f27e23738e0>
requirements,(Empty list)


[0] <PropertyValueGroup 'Power' (1e241678-282c-46f3-abbd-e0cf34e743ee)>
[1] <PropertyValueGroup 'Power Level' (a0cb5a23-955e-43d6-a633-2c4e66991364)>

'Power'

[0] <FloatPropertyValue 'Min' (4c5eba40-fcc3-480f-b88f-e375dadc00e5)>
[1] <FloatPropertyValue 'Max' (196467a2-ecee-4f35-8987-6a898cd34616)>

'Power Level'

[0] <EnumerationPropertyValue 'Skill' (7367c731-aabb-4807-9d74-7ef2c6482ec9)>

The domain scope (i.e. what ArchitectureLayer) this extension can be apploed to is not yet processed further. Scopes have rules underneath to control on which ModelObject's this can be applied to.
This is not yet supported.

In [6]:
level = darkmagic.groups[-1]
level.description

Markup('[PROPERTY]DarkMagic.Power.Max<10000.0[/PROPERTY]\n[ARCHITECTURE]LOGICAL[/ARCHITECTURE]')

## Access PVMT property values

The ground rule is that you can access PVMT via the `.pvmt` attribute on literally everything you can find in a model. Then the pattern is `domain.group.subgroup.[moresubgroups...].property` from which the pvmt can be indexed with:

Let's check Dumbledore's dark magic power level:

In [7]:
dumbledore = model.la.all_components.by_name("Prof. A. P. W. B. Dumbledore")
dumbledore.pvmt["DarkMagic.Power Level.Skill"]

'Mighty'

In our test model we have more famous wizzards. Let us define a function to collect all property values and load them into a pandas.DataFrame:

In [8]:
import pandas as pd


def get_darkmagic_dataframe(model: capellambse.MelodyModel) -> pd.DataFrame:
    """Return a DataFrame with the darkmagic power for all LogicalActors."""
    return pd.DataFrame(
        [
            {
                "Minimum Power": actor.pvmt["DarkMagic.Power.Min"],
                "Maximum Power": actor.pvmt["DarkMagic.Power.Max"],
                "Skill level": actor.pvmt["DarkMagic.Power Level.Skill"],
            } for actor in model.la.all_actors
        ], index=(actor.name for actor in model.la.all_actors)
    )


get_darkmagic_dataframe(model)

,Minimum Power,Maximum Power,Skill level
Voldemort,NaN,10000.0,TheOneWhoShallNotBeNamed
Harry J. Potter,300.0,500.0,Apprentice
Prof. A. P. W. B. Dumbledore,800.0,1600.0,Mighty
R. Weasley,50.0,150.0,Apprentice
Prof. S. Snape,200.0,450.0,Mighty


## Writing Property values

In [9]:
dumbledore.pvmt["DarkMagic.Power.Min"] = 1000.0

TypeError: 'PropertyValueProxy' object does not support item assignment